Unemployed worker's problem

In [1]:
# Import libraries

import numpy as np
# import matplotlib.pyplot as plt
from numba import njit, float64
from numba.experimental import jitclass

In [2]:
search_data = [
    ('b', float64),      # unemployment compensation
    ('beta', float64),   # discount factor
    ('gamma', float64),  # risk aversion
    ('w', float64[:]),   # array of wage values, w[i] = wage at state i
    ('q', float64[:])    # array of probabilities
]

In [3]:
# Speificy parameter values

b0 = 0.5 #employment benefit
b1 = 0.8


In [4]:
### Exogenous Wage Distribution
n = 3_000
w_min, w_max = 1, 2

q_default = np.ones(n+1,dtype=float) / (n+1)      # Uniform distribution  

w_default = np.linspace(w_min, w_max, n+1)

In [5]:
@jitclass(search_data)
class SearchModel:

    def __init__(self, 
                b = b0,      
                beta = 0.9,
                gamma = 2.0, # For linear utility set gamma= 0.0
                w=w_default, q=q_default):

        self.b, self.beta, self.gamma = b, beta, gamma
        self.w, self.q = w_default, q_default

    # CRRA utility:
    def u(self, x):
        return (x**(1-self.gamma) - 1.0)/(1-self.gamma) 
        
    # Inverse of utility
    def u_inv(self, y):
        return (y*(1-self.gamma) + 1)**(1/(1-self.gamma)) 
    

    def state_action_values(self, i, v):
        """
        The values of state-action pairs.
        """
        # Simplify names
        b, beta, w, q, u = self.b,self.beta, self.w, self.q, self.u
        # Evaluate value for each state-action pair
        # Consider action = accept or reject the current offer
        Expect = np.sum(v * q)
        accept = u(w[i]) / (1 -  beta) 
        reject = u(b) + beta * Expect

        return np.array([accept, reject])

In [6]:
@njit
def compute_reservation_wage(model,
                             max_iter=500,
                             tol=1e-6):

    # Simplify names
    b, beta, w, q = model.b, model.beta, model.w, model.q
    u, u_inv = model.u, model.u_inv

    # value function 
    n = len(w)
    v = u(w) / (1 - beta)        # initial guess
    v_next = np.empty_like(v)
    i = 0
    error = tol + 1
    while i < max_iter and error > tol:

        for iw in range(n):
            v_next[iw] = np.max(model.state_action_values(iw, v))

        error = np.max(np.abs(v_next - v))
        i += 1

        v[:] = v_next  # copy contents into v

    # reservation wage
    expect = np.sum(v * q)
    what = u_inv( (1-beta)*(u(b) + beta * expect))
    return what


In [7]:
print("Reservation wage is {:.3f}".format(compute_reservation_wage(SearchModel(b = b0))))

Reservation wage is nan


In [8]:
sm0 = SearchModel(b=b0)

In [9]:
%%timeit -r 3 -n 10
result = compute_reservation_wage(sm0)

10 ms ± 166 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)
